<a href="https://colab.research.google.com/github/Maryemt/Gen-Assoc/blob/develop/Devoir_Marieme_TOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webscrapping sur les données de voitures

Master I DIT - Session Janvier 2024 - Cours Data Collection

Ndeye Marieme TOP

**Instructions**:

Sur le site https://sn.coinafrique.com/categorie/voitures

Récupérer sur **5 pages** les informations suivantes:
- Constructeur
- Modele
- Kilométrage
- Transmission
- Carburant
- Prix
- Adresse

# Import needed packages

In [13]:
# Import packages

import pandas as pd
from requests import get #  get a page html code
from bs4 import BeautifulSoup as bs #  store html code in a bs object


#Test principle step by step

The purpose is to test how to access the data

In [14]:
# url
url = 'https://sn.coinafrique.com/categorie/voitures'

res = get(url)
soup = bs(res.text, 'html') # store code in bs obj
containers = soup.find_all('div', class_ ='card-content ad__card-content')  # div balises containing car informations

# Check containers length
#len(containers)

# Start with first container
container = containers[0]
container



<div class="card-content ad__card-content"><div><p class="ad__card-price"><a data-is-top-ad-link="top" data-view-origin-parent-tree="6" href="/annonce/voitures/peugeot-2008-2015-4696931" title="Peugeot 2008 2015">7 000 000<span>CFA</span></a></p><p class="ad__card-description"><a data-is-top-ad-link="top" data-view-origin-parent-tree="6" href="/annonce/voitures/peugeot-2008-2015-4696931" title="Peugeot 2008 2015">Peugeot 2008 2015</a></p><p class="ad__card-location"><i class="material-icons">location_on</i><span>Dakar, Sénégal</span></p></div><div class="card-action ad__card-action is-pro"><a class="pro-contact-card" data-ad-category="Voitures" data-ad-title="Peugeot 2008 2015" data-post-id="4696931" href="/profil/d74c50cb-2928-44ef-8206-782eab547bc6" title="Visiter le profil de LYMA Auto"><div class="profile-picture"><img alt="photo utilisateur" class="pro-icon circle profile-card__img" src="https://coinafrique-ads-photos.s3.amazonaws.com/photos/43118871-6cd.png"/><p class="title">LYM

In [15]:
pre_link = container.find('a')['href']
pre_link

'/annonce/voitures/peugeot-2008-2015-4696931'

In [16]:
# Concatenate with prefix
link = 'https://sn.coinafrique.com' + pre_link
link

'https://sn.coinafrique.com/annonce/voitures/peugeot-2008-2015-4696931'

In [17]:
# get html code of item page
res_c = get(link)
bsoup_c = bs(res_c.text, 'html.parser')
# bsoup_c


### Get informations one by one

In [18]:
# Balise contenant info suivantes:
info = bsoup_c.find_all('div', class_="details-characteristics")[0].find_all('span')

constructeur = info[1].text
modele = info[3].text
kilometrage = info[5].text.replace(' km','')
transmission = info[7].text
carburant = info[9].text

# prix et adresse
prix = bsoup_c.find_all('p', class_="price")[0].text.replace(' CFA','').replace(' ','')
adresse = bsoup_c.find_all('div', class_="row valign-wrapper extra-info-ad-detail")[0].find_all('span')[2].text.replace(',','')


# Scrap data of one page

In [19]:
# Scraper les données de la page

data = [] # List to store informations

for container in containers:

  # get link to the car page
  link = 'https://sn.coinafrique.com'+ container.find('a')['href']
  # get html code de la page de l'item
  res_c = get(link)
  bsoup_c = bs(res_c.text, 'html.parser')

  try:
    # Balise contenant info suivantes:
    info = bsoup_c.find_all('div', class_="details-characteristics")[0].find_all('span')
    constructeur = info[1].text
    modele = info[3].text
    kilometrage = info[5].text.replace(' km','')
    transmission = info[7].text
    carburant = info[9].text
    # prix et adresse
    prix = bsoup_c.find_all('p', class_="price")[0].text.replace(' CFA','').replace(' ','')
    adresse = bsoup_c.find_all('div', class_="row valign-wrapper extra-info-ad-detail")[0].find_all('span')[2].text.replace(',','')


    dico = {
        'constructeur' : constructeur,
        'modele' : modele,
        'kilometrage' : kilometrage,
        'transmission' : transmission,
        'carburant': carburant,
        'prix' : prix,
        'adresse' : adresse
    }
    data.append(dico)

  except:
    pass

DF = pd.DataFrame(data)


In [20]:
DF.shape

(56, 7)

In [21]:
DF.head()

,constructeur,modele,kilometrage,transmission,carburant,prix,adresse
0,Peugeot,2008,0,Automatique,Gasoil,7000000,Dakar Sénégal
1,Rover,range rover,0,Automatique,Essence,29500000,Dakar Sénégal
2,Mitsubishi,rvr,120000,Automatique,Essence,6300000,Dakar Sénégal
3,Volkswagen,Polo,171000,Manuel,Gasoil,2400000,Fass Dakar Sénégal
4,Toyota,corolla,235300,Manuel,Essence,2900000,Grand Yoff Dakar Sénégal


# Generalisation

In [28]:
df = pd.DataFrame()

for p in range(1,6):      # 6 wont be considered

  # URL of site's pages
  url = f'https://sn.coinafrique.com/categorie/voitures?page={p}'

  # Get html code of the current page and create containers
  res = get(url)
  soup = bs(res.text, 'html') # store code in bs obj

  # All car informations in <div>
  containers = soup.find_all('div', class_ ='card-content ad__card-content')

  # See progress
  print("Page", p , "| Number of items : ",len(containers))

  data = []
  for container in containers:

    # get link to the car page : need to add the website link as a prefix
    link = 'https://sn.coinafrique.com'+ container.find('a')['href']

    # get html code of item
    res_c = get(link)
    bsoup_c = bs(res_c.text, 'html.parser')

    try:

      info = bsoup_c.find_all('div', class_="details-characteristics")[0].find_all('span')
      constructeur = info[1].text
      modele = info[3].text
      kilometrage = info[5].text.replace(' km','')
      transmission = info[7].text
      carburant = info[9].text
      prix = bsoup_c.find_all('p', class_="price")[0].text.replace(' CFA','').replace(' ','')
      adresse = bsoup_c.find_all('div', class_="row valign-wrapper extra-info-ad-detail")[0].find_all('span')[2].text.replace(',','')

      dico = {
          'constructeur' : constructeur,
          'modele' : modele,
          'kilometrage' : kilometrage,
          'transmission' : transmission,
          'carburant': carburant,
          'prix' : prix,
          'adresse' : adresse
      }
      data.append(dico)

    except:
      pass

  DF = pd.DataFrame(data)
  print(DF.shape)
  df = pd.concat([df, DF]).reset_index(drop = True)

  #DF = pd.concat([DF, pd.DataFrame(data)]).reset_index(drop = True)


Page 1 | Number of items :  56
(56, 7)
Page 2 | Number of items :  41
(41, 7)
Page 3 | Number of items :  53
(53, 7)
Page 4 | Number of items :  37
(37, 7)
Page 5 | Number of items :  49
(49, 7)


In [29]:
df.shape

(236, 7)

In [31]:
df.head()

,constructeur,modele,kilometrage,transmission,carburant,prix,adresse
0,Ford,focus,0,Automatique,Essence,2700000,Dakar Sénégal
1,Peugeot,2008,0,Automatique,Gasoil,7000000,Dakar Sénégal
2,Rover,range rover,0,Automatique,Essence,29500000,Dakar Sénégal
3,Mitsubishi,rvr,120000,Automatique,Essence,6300000,Dakar Sénégal
4,Volkswagen,Polo,171000,Manuel,Gasoil,2400000,Fass Dakar Sénégal


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   constructeur  236 non-null    object
 1   modele        236 non-null    object
 2   kilometrage   236 non-null    object
 3   transmission  236 non-null    object
 4   carburant     236 non-null    object
 5   prix          236 non-null    object
 6   adresse       236 non-null    object
dtypes: object(7)
memory usage: 13.0+ KB


If you want to save the data

In [33]:
df.to_excel('car_data.xlsx', index=False)